In [ ]:
import sys,os
import glob
import torch,dill
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from iresnet import iResBlock

In [ ]:
glob.glob(os.path.expanduser('~/tb-experiments/iresnet_singular/**/*.trainer'),recursive=True)

In [ ]:
p0 = glob.glob(os.path.expanduser('~/tb-experiments/iresnet_singular/**/*.trainer'),recursive=True)[-1]
trainer = torch.load(p0,pickle_module=dill)
trainer.model.eval();

In [ ]:
def change_tol(module):
    if isinstance(module,iResBlock):
        module.inverse_tol=1e-5

In [ ]:
for mb in trainer.dataloaders['dev']:
    break
x=mb[0]
#trainer.model.apply(change_tol)
with torch.no_grad():
    inp = trainer.model[1].foot(x)

In [ ]:
with torch.no_grad():
    output = trainer.model[1].body(inp)

In [ ]:
with torch.no_grad():
    inp_reconstructed = trainer.model[1].body.inverse(output)

In [ ]:
torch.randn_like(x[0])

In [ ]:
((inp_reconstructed[0] - inp[0]).norm()/(inp[0]).norm())

In [ ]:
plt.hist(inp_reconstructed[0].cpu().data.numpy().reshape(-1),alpha=.5)
plt.hist(inp[0].cpu().data.numpy().reshape(-1),alpha=.5)

In [ ]:
#print(y.shape)
grid = vutils.make_grid(mb[0]).cpu().data.numpy().transpose(1,2,0)/2
plt.imshow(grid)

In [ ]:
for zi in z: print(zi.shape)